# Model to detect if a Starbucks Customer is passive or not, based on their reponse to offers

In [347]:
import pandas as pd
import datetime,json
import warnings
warnings.filterwarnings('ignore')

# Load and Clean Data

# 1. Portfolio Data - Data about the rewards/offers programs

In [299]:
#Load portfolio data 
portfolio = pd.read_json('portfolio.json', orient='records',lines=True)
portfolio.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [300]:
#View tbale information
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 608.0+ bytes


In [301]:
#Shape of portfolio table
portfolio.shape

(10, 6)

In [302]:
#Changing column 'id' to 'offer_id' as 'id' is used on other tables
portfolio.rename(columns = {'id': 'offer_id'}, inplace = True)

In [303]:
#check for null values
portfolio.isnull().sum()

reward        0
channels      0
difficulty    0
duration      0
offer_type    0
offer_id      0
dtype: int64

There is no missing data

In [304]:
#check for duplicate rows:
d = portfolio[portfolio.duplicated('offer_id')]
print(d)

Empty DataFrame
Columns: [reward, channels, difficulty, duration, offer_type, offer_id]
Index: []


There are no duplicate ids

# 2. Profile Data - Customer info

In [305]:
#Load profile data
profile = pd.read_json('profile.json', lines=True)
profile.head()

,gender,age,id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [306]:
#dataset information
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [307]:
#Shape of Profile table
profile.shape

(17000, 5)

In [308]:
#Changing column id to offer_id as 'id' is used on other tables
profile.rename(columns = {'id': 'cust_id'}, inplace = True)

In [309]:
#check for null values
profile.isnull().sum()

gender              2175
age                    0
cust_id                0
became_member_on       0
income              2175
dtype: int64

There are 2175 rows that ha null values for gender and income data.Let's see if those records are valid or not.

In [310]:
missing_profile = profile[profile.gender.isnull() & profile.income.isnull()]
missing_profile

,gender,age,cust_id,became_member_on,income
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
4,None,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN
6,None,118,8ec6ce2a7e7949b1bf142def7d0e0586,20170925,NaN
7,None,118,68617ca6246f4fbc85e91a2a49552598,20171002,NaN
...,...,...,...,...,...
16980,None,118,5c686d09ca4d475a8f750f2ba07e0440,20160901,NaN
16982,None,118,d9ca82f550ac4ee58b6299cf1e5c824a,20160415,NaN
16989,None,118,ca45ee1883624304bac1e4c8a114f045,20180305,NaN
16991,None,118,a9a20fa8b5504360beb4e7c8712f8306,20160116,NaN


We can see that all the rows where gender and salary values are null have invalid age values as 118. Since these are invalid entries with no customer information available, we can remove it from the data.

In [311]:
#Drop all rows with Nan values having invalid entries
profile.dropna(axis=0,inplace=True)
profile.head()

,gender,age,cust_id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,20180426,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,20180209,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,20171111,51000.0


In [312]:
#Print shape of Profile table after removing invalid customer entries
profile.shape

(14825, 5)

In [313]:
#Changing 'became_member_on' from int64 to date type
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'].astype(str), format='%Y%m%d')
profile.head()

,gender,age,cust_id,became_member_on,income
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0


In [314]:
#Add column to estimate the number of days since the customer became a member
profile['days_as_member'] = (datetime.datetime.today() - profile['became_member_on']).dt.days
profile.head()

,gender,age,cust_id,became_member_on,income,days_as_member
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,1220
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,1287
5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,935
8,M,65,389bc3fa690240e798340f5a15918d5c,2018-02-09,53000.0,1011
12,M,58,2eeac8d8feae4a8cad5a6af0499a211d,2017-11-11,51000.0,1101


# 3. Transcript Data - Has Information on Customer behaviour on offers

In [315]:
#Load transcipt data - Deatils about Customer behaviour on the offers received
transcript = pd.read_json('transcript.json', lines=True)
transcript.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [316]:
#View transcript table data
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [317]:
#Print shape of the table
transcript.shape

(306534, 4)

In [318]:
#Rename 'person' to 'cust_id' to match with profile table
transcript.rename(columns = {'person':'cust_id'}, inplace=True)
transcript.sample(2)

,cust_id,event,value,time
274505,91a5f86f02964fa5b6a41b531547815d,transaction,{'amount': 12.45},600
141485,26e8ef90d9d64c919ebbe964781c122f,offer completed,{'offer_id': '2906b810c7d4411798c6938adc9daaa5...,372


In [319]:
#Check for NaN values
transcript.isnull().sum()

cust_id    0
event      0
value      0
time       0
dtype: int64

There is no missing data!

In [320]:
#Print different event types and count
transcript['event'].value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

In [321]:
#add convert 'time' field in hours to days
transcript['days'] = transcript['time']/24
transcript.sample(5)

,cust_id,event,value,time,days
274356,10df92b3876142e8a6745c86276057e7,transaction,{'amount': 23.21},600,25.00
47876,379c3d188896452bb3ce05c91801e191,transaction,{'amount': 30.66},132,5.50
184983,11d7a9296e35432f8a34b21e337529fb,transaction,{'amount': 33.48},450,18.75
73044,1541564cf2d14ec0ab27872f19acc4ba,offer viewed,{'offer id': 'ae264e3637204a6fb9bb56bc8210ddfd'},180,7.50
36013,fc05227412c64551b11d24e6a07332c1,transaction,{'amount': 14.92},72,3.00


In [322]:
transcript['value'].head()

0    {'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1    {'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2    {'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3    {'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4    {'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}
Name: value, dtype: object

In [323]:
#since the 'values' column is dict type, we can parse the data as key value pairs. 
#'value-type' field will contain offer/amount info 
#'values' field will hold the corresponding offer-id or transaction amount
transcript['value_type'] = transcript['value'].apply(lambda x: list(x.keys())[0])
transcript['values'] = transcript['value'].apply(lambda x: list(x.values())[0])

In [324]:
#Drop 'value' column
transcript.drop('value', axis=1, inplace=True)

In [325]:
transcript.sample(5)

,cust_id,event,time,days,value_type,values
29234,f024f2a55f4f4f578216377635c9c704,offer completed,42,1.75,offer_id,2298d6c36e964ae4a3e7e9706d1fb8c2
22075,e2880c9e137a45358660085f5ced186f,transaction,18,0.75,amount,15.34
264859,d56d701948d7473d80c0583ca8221eef,offer viewed,582,24.25,offer id,fafdcd668e3743c1bb461111dcafc2a4
125542,2b0e698462194cd787308c7e2a7a8bfd,transaction,336,14.00,amount,16.89
213851,7f49a11dda5b44c2b6afb7d3303c2a88,offer received,504,21.00,offer id,fafdcd668e3743c1bb461111dcafc2a4


In [326]:
transcript['value_type'].value_counts()

amount      138953
offer id    134002
offer_id     33579
Name: value_type, dtype: int64

We see two names 'offer id' and offer_id' both of which are the same. Lets replace it to one value.

In [327]:
transcript['value_type'].replace('offer id', 'offer_id',inplace=True )

In [328]:
transcript['value_type'].value_counts()

offer_id    167581
amount      138953
Name: value_type, dtype: int64

In [331]:
transcript.sample(5)

,cust_id,event,time,days,value_type,values
153145,ec370a6f13d64a7fa3636c64a4c87781,offer received,408,17.00,offer_id,0b1e1539f2cc45b7b9fa7c272da2e1d7
95896,71d2e0a7ae8942ac9ad23b5e5db04ddd,transaction,252,10.50,amount,29.37
24476,4d20573b5dee49718990534338be745e,transaction,30,1.25,amount,1.49
117707,558b5f855ed1445c85a65dd2e55e86e3,offer received,336,14.00,offer_id,0b1e1539f2cc45b7b9fa7c272da2e1d7
38028,746e444adffa448587530c0d57729c1b,transaction,78,3.25,amount,40.11


In [352]:
#Splitting into transaction and offers dataframes
offers = transcript[transcript['value_type'] == 'offer_id']
offers.drop(columns=['value_type'], axis=1, inplace=True)
offers.rename(columns = {'values': 'offer_id'}, inplace=True)
offers.sample(5)

,cust_id,event,time,days,offer_id
207555,d0ab05be7544435f935f13082b31ff0e,offer received,504,21.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
119278,3ee1c588015f405eb0cc8aea95fd9daa,offer received,336,14.0,0b1e1539f2cc45b7b9fa7c272da2e1d7
248953,b10f5a1006c249dca0faa1c4cbb54322,offer received,576,24.0,5a8bc65990b245e5a138643cd4eb9837
209417,e0780fee1ddf45f98fcc65beb58966a1,offer received,504,21.0,9b98b8c7a33c4b65b9aebfe6a799e6d9
210721,e2e0fce58b6d4c239a69ab22f4e6fd58,offer received,504,21.0,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [351]:
transactions = transcript[transcript['value_type'] == 'amount']
transactions.drop(columns=['value_type'], axis=1, inplace=True)
transactions.rename(columns = {'values': 'amount'}, inplace=True)
transactions.sample(5)

,cust_id,event,time,days,amount
199238,910a58abd30c4df596b3904b4471c626,transaction,492,20.5,14.49
101234,93cab11de46545cb918d43af7f015ffb,transaction,276,11.5,4.44
188495,fe9087e80c4744bead50ab4f92a32d96,transaction,456,19.0,20.79
105039,46cae479b56c479597ff23bbcdc95693,transaction,300,12.5,3.48
303178,acfbd718a7a848698a1ac398e8566179,transaction,696,29.0,17.3
